In [2]:
from collections import defaultdict
from attrs import define
import numpy as np
from tqdm import tqdm

In [204]:
def load_data(
    path,
):
    with open(path, "r") as f:
        lines = f.readlines()
        dimension = int(lines[3].split(":")[1])
        capacity = int(lines[5].split(":")[1])
        idx = 7
        graph = [None] * dimension
        while len(lines[idx].strip().split(" ")) == 3:
            node, x, y = lines[idx].strip().split(" ")
            graph[int(node) - 1] = (int(x), int(y))
            idx += 1

        graph = np.array(graph)

        idx += 1
        needs = np.zeros(dimension, dtype=np.int32)
        while len(lines[idx].strip().split(" ")) == 2:
            node, cap = lines[idx].strip().split(" ")
            needs[int(node) - 1] = int(cap)
            idx += 1

    return graph, needs, dimension, capacity
graph, needs, dimension, capacity = load_data("data/E/E-n23-k3.vrp")

In [18]:
def distance(coords):
    return np.linalg.norm(coords, axis=-1)

In [161]:
# Calculate the euclidian distance in n-space of the route r traversing cities c, ending at the path start.
path_distance = lambda r,c: np.sum([np.linalg.norm(c[r[p]]-c[r[p-1]]) for p in range(len(r))])
# Reverse the order of all elements from element i to element k in array r.
two_opt_swap = lambda r,i,k: np.concatenate((r[0:i],r[k:-len(r)+i-1:-1],r[k+1:len(r)]))

def TSP_loop(
    graph,
    path,
    initial_node = 0,
    improvement_threshold=0.001
):
    cities = np.vstack((graph[path], graph[0]))

    route = np.arange(cities.shape[0]) # Make an array of row numbers corresponding to cities.
    improvement_factor = 1 # Initialize the improvement factor.
    best_distance = path_distance(route,cities) # Calculate the distance of the initial path.
    while improvement_factor > improvement_threshold: # If the route is still improving, keep going!
        distance_to_beat = best_distance # Record the distance at the beginning of the loop.
        for swap_first in range(1,len(route)-2): # From each city except the first and last,
            for swap_last in range(swap_first+1,len(route)): # to each of the cities following,
                new_route = two_opt_swap(route,swap_first,swap_last) # try reversing the order of these cities
                new_distance = path_distance(new_route,cities) # and check the total distance with this modification.
                if new_distance < best_distance: # If the path distance is an improvement,
                    route = new_route # make this the accepted best route
                    best_distance = new_distance # and update the distance corresponding to this route.
        improvement_factor = 1 - best_distance/distance_to_beat # Calculate how much the route has improved.
    return path[route[route != cities.shape[0] -1]] # When the route is no longer improving substantially, stop searching and return the route.

# route = np.array([1, 2, 3, 4, 22, 31], dtype=np.int32)
# print(calc_path_metric(graph, route))
# route = TSP_loop(graph, np.array([1, 2, 3, 4, 22, 31]), 0.001)
# print(calc_path_metric(graph, route))

# output: 
# 343.1853049674675
# 281.23099883169414

In [162]:
def random_construct_loop(
    graph, 
    needs, 
    nodes_pool, # pool клиентов еще не обслуженных
    capacity,
    initial_node = 0,
):
    assert initial_node not in nodes_pool
    nodes_pool = np.array(nodes_pool)
    np.random.shuffle(nodes_pool)

    current_capacity = 0
    # print(needs.shape[0])
    needs = np.cumsum(needs[nodes_pool]) < capacity
    return TSP_loop(graph, nodes_pool[needs])

In [163]:
def construct_gene(
    graph, 
    needs, 
    capacity,
    initial_node = 0,
    dimension = None,
    construct_loop = random_construct_loop,
): 
    if dimension is None:
        dimension = graph.shape[0]
    nodes_pool = np.arange(1, dimension)
    gene = []
    while nodes_pool.shape[0] != 0:
        gene.append(construct_loop(graph, needs, nodes_pool, capacity, initial_node))
        nodes_pool = nodes_pool[~np.in1d(nodes_pool,gene[-1])]
    return gene

In [164]:
def random_mutation(
    gene, 
    graph, 
    needs, 
    capacity,
    alpha=0.8,
    initial_node = 0,
    dimension = None,
    construct_loop = random_construct_loop,
):
    fraction = alpha # в контексте данного подхода к мутации - 
    # альфа это часть гена, которая пересоберется заного

    # fully recreate part of gene
    # количество изменяемых генов семплируем из 
    # експоненциального распределения
    # с минимум 2 пермутациями
    p = alpha * np.exp(-alpha * np.arange(len(gene) - 2))
    p = p / p.sum()
    size_recr = np.random.choice(len(gene) - 2, size=1, p=p)[0] + 2

    # size_recr = int(fraction * len(gene))
    # p = np.exp(-np.arange(len(gene) - size_recr))
    # p[0] = 1 - p.sum()
    # size_recr = np.random.choice(np.arange(size_recr, len(gene)), size=1, p=p)
    # print(size_recr, len(gene))
    # size_recr = size_recr if size_recr > 1 else 2

    genes_reassemble = np.random.choice(len(gene), replace=False, size=size_recr)
    nodes_pool = np.hstack([gene[gr] for gr in genes_reassemble])
    gene = [gene[gr] for gr in range(len(gene)) if gr not in genes_reassemble]
    while nodes_pool.shape[0] != 0:
        gene.append(construct_loop(graph, needs, nodes_pool, capacity, initial_node))
        nodes_pool = nodes_pool[~np.in1d(nodes_pool,gene[-1])]
    return gene

In [165]:
def random_crossingover(
    gene1, 
    gene2,
    graph,
    dimension,
    capacity,
    needs,
    initial_node = 0,
    construct_loop = random_construct_loop,
):
    # оба гена вносят равный вклад, и остаток - случайно заполняется
    nodes_pool = np.arange(1, dimension)
    new_gene = []
    idxs = [0, 0]
    merging = 0
    genes = [gene1, gene2]
    while idxs[0] < len(gene1) and idxs[1] < len(gene2):
        if np.in1d(genes[merging][idxs[merging]], nodes_pool).all():
            new_gene.append(genes[merging][idxs[merging]])
            nodes_pool = nodes_pool[~np.in1d(nodes_pool,new_gene[-1])]
            idxs[merging] += 1
            merging = 1 if merging == 0 else 0
        else:
            idxs[merging] += 1
    
    merging = 1 if merging == 0 else 0
    while idxs[merging] < len(genes[merging]):
        if np.in1d(genes[merging][idxs[merging]], nodes_pool).all():
            new_gene.append(genes[merging][idxs[merging]])
            nodes_pool = nodes_pool[~np.in1d(nodes_pool,new_gene[-1])]
        idxs[merging] += 1

    while nodes_pool.shape[0] != 0:
        new_gene.append(construct_loop(graph, needs, nodes_pool, capacity, initial_node))
        nodes_pool = nodes_pool[~np.in1d(nodes_pool, new_gene[-1])]
    return new_gene

In [166]:
gene1 = construct_gene(
    graph, 
    needs, 
    capacity,
)
gene2 = construct_gene(
    graph, 
    needs, 
    capacity,
)

res = random_crossingover(
    gene1, 
    gene2,
    graph,
    dimension,
    capacity,
    needs,
)

res

[array([16, 25,  9, 19,  4, 24]),
 array([32, 27, 29, 26,  5, 39]),
 array([33, 41, 21,  3,  1]),
 array([ 2, 17, 12, 13, 42]),
 array([44, 35, 28, 30, 34, 20, 40]),
 array([14, 10, 31, 38,  7]),
 array([43, 11, 36,  6, 18, 15, 23, 37,  8, 22])]

In [167]:
def calc_path_metric(
    graph, 
    path_nodes,
    initial_node = 0,
):
    assert initial_node not in path_nodes
    path_distance = distance(graph[path_nodes[0]] - graph[initial_node]) + \
               distance(graph[path_nodes[-1]] - graph[initial_node])
    path_distance += np.sum(
        np.sqrt(
            np.sum(
                np.square(
                    graph[path_nodes[:-1]] - graph[path_nodes[1:]]
                ), axis=1
            )
        )
    )
    return path_distance

def calc_gene_metric(
    graph,
    gene, # array of paths
    initial_node = 0,
):
    gene_metric = 0
    for path in gene:
        gene_metric += calc_path_metric(graph, path, initial_node)
    return gene_metric


In [168]:
def sampling_selection(
    graph,
    population,
    sample_size,
    initial_node = 0,
    temperature = 0.7,
    max_fitness_scale = 30,
): 
    fitness_vector = np.array([calc_gene_metric(graph, gene, initial_node=initial_node) for gene in population])
    fitness_vector_probs = fitness_vector - fitness_vector.min() + 1 # чтобы / 0 не было
    fitness_vector_probs = - fitness_vector_probs / fitness_vector_probs.max() * max_fitness_scale
    # fitness_vector_probs = 1 / fitness_vector_probs
    fitness_vector_probs = np.exp(fitness_vector_probs / temperature)
    fitness_vector_probs = fitness_vector_probs / fitness_vector_probs.sum()
    selection = np.random.choice(fitness_vector.shape[0], size=int(sample_size * len(population)), p=fitness_vector_probs)
    return [population[sel] for sel in selection], fitness_vector[selection]

In [169]:
def cross_population(
    graph,
    population,
    new_population_size,
    fitness_vector,
    dimension,
    capacity,
    needs,
    initial_node = 0,
    temperature = 0.7,
    construct_loop = random_construct_loop,
    cross_function = random_crossingover,
    max_fitness_scale = 30,
):
    fitness_vector_probs = fitness_vector - fitness_vector.min() + 1 # чтобы / 0 не было
    fitness_vector_probs = - fitness_vector_probs / fitness_vector_probs.max() * max_fitness_scale
    # fitness_vector_probs = 1 / fitness_vector_probs
    fitness_vector_probs = np.exp(fitness_vector_probs / temperature)
    fitness_vector_probs = fitness_vector_probs / fitness_vector_probs.sum()
    # print(np.partition(fitness_vector_probs, len(population) - 10)[len(population) - 10:])
    population_income = []
    while len(population_income) < new_population_size - len(population) :
        cross_genes_idx = np.random.choice(len(population), size=2, replace=False, p=fitness_vector_probs)
        population_income.append(cross_function(
            population[cross_genes_idx[0]],
            population[cross_genes_idx[1]],
            graph,
            dimension,
            capacity,
            needs,
            initial_node = 0,
            construct_loop = construct_loop,
        ))
    return population + population_income

In [195]:
def greedy_loop(
    graph, 
    needs, 
    nodes_pool, # pool клиентов еще не обслуженных
    capacity,
    proximity_metric,
    initial_node = 0,
    temperature = 0.5
):
    proximity_metric = proximity_metric[nodes_pool]
    proximity_metric = proximity_metric / proximity_metric.max() * 40
    proximity_metric = np.exp(proximity_metric / temperature)
    proximity_metric = proximity_metric / proximity_metric.sum()
    node_optimize = np.random.choice(nodes_pool, size=1, p=proximity_metric)[0]
    initial_hub = graph[initial_node] - graph[node_optimize]
    distances = distance(graph[nodes_pool] - initial_hub)
    cur_cap = needs[node_optimize]
    path = [node_optimize]
    sorted_dist = np.argsort(distances)
    for node in nodes_pool[sorted_dist]:
        if node != node_optimize:
            if cur_cap + needs[node] < capacity:
                cur_cap += needs[node]
                path.append(node)

    return TSP_loop(graph, np.array(path))

def population_initialization(
    graph, 
    needs,
    pop_size,
    dimension,
    capacity,
    initial_node = 0,
):
    # Можно разумно инициализировать популяцию
    population = []
    client_proximity_metric = distance(graph - graph[initial_node]) * needs
    for _ in range(pop_size):
        nodes_pool = np.arange(1, dimension)
        gene = []
        while nodes_pool.shape[0] != 0:
            gene.append(
                greedy_loop(
                    graph, 
                    needs, 
                    nodes_pool, # pool клиентов еще не обслуженных
                    capacity,
                    client_proximity_metric,
                    initial_node = initial_node,
                )
            )
            nodes_pool = nodes_pool[~np.in1d(nodes_pool,gene[-1])]
        population.append(gene)

    return population

In [205]:
from tqdm import tqdm

def calc_best_solution(
    data_path = "data/B/B-n31-k5.vrp",
    population_size = 100,
    evolution_steps = 100,
    mutation_factor = 0.2,
    dieback_rate = 0.8,
    temperature = 0.2,
    crossingover = random_crossingover,
    construct_loop = random_construct_loop,
    mutation = random_mutation,
    use_tqdm=False,
):
    graph, needs, dimension, capacity = load_data(data_path)

    # generate population
    population = population_initialization(
        graph, 
        needs,
        population_size,
        dimension,
        capacity,
        initial_node = 0,
    )
    best_solution = (population[0], calc_gene_metric(graph, population[0]))

    pbar = range(evolution_steps)
    if use_tqdm:
        pbar = tqdm(pbar)
    for eval_steps in pbar:
        # mutate
        population = [
            mutation(
            gene, 
            graph, 
            needs, 
            capacity,
            alpha=mutation_factor,
            construct_loop = construct_loop,
        ) for gene in population]
        
        # select
        population, fit = sampling_selection(
            graph,
            population,
            sample_size=dieback_rate,
            temperature = temperature,
        )
        if use_tqdm:
            pbar.set_postfix_str(f"best fit: {np.min(fit)}")
        # remember best solution
        if best_solution[1] > fit.min():
            fit_min = np.argmin(fit)
            best_solution = (population[fit_min], fit[fit_min]) 

        # cross over
        population = cross_population(
            graph,
            population,
            population_size,
            fit,
            dimension,
            capacity,
            needs,
            temperature = temperature,
            construct_loop = construct_loop,
            cross_function = crossingover,
        )

    return best_solution

calc_best_solution(
    data_path = "data/E/E-n23-k3.vrp",
    population_size = 200,
    evolution_steps = 15,
    mutation_factor = 0.2,
    dieback_rate = 0.8,
    temperature = 0.2,
    crossingover = random_crossingover,
    construct_loop = random_construct_loop,
    mutation = random_mutation,
    use_tqdm=True,
)

In [197]:
from pathlib import Path

def dump_solution(solution, score, file_name):
    # Route #1: 21 31 19 17 13 7 26
    # Route #2: 12 1 16 30
    # Route #3: 27 24
    # Route #4: 29 18 8 9 22 15 10 25 5 20
    # Route #5: 14 28 11 4 23 3 2 6
    # Cost 784

    file_lines = []
    for idx, sol in enumerate(solution):
        file_lines.append(f'Route #{idx + 1}: {" ".join(map(str, sol))}\n')
    file_lines.append(f"Cost {int(score)}")
    with open(file_name, "w") as f:
        f.writelines(file_lines)

pbar = tqdm(Path("data/").rglob("*.vrp"), total = len(list(Path("data/").rglob("*.vrp"))))
for file_vrp in pbar:
    pbar.set_postfix_str(str(file_vrp))
    if not file_vrp.with_suffix(".msol").exists():
        try:
            # Возможно формат некоторых E файлов не подойдет для парсинга
            solution, score = calc_best_solution(
                data_path = str(file_vrp),
                population_size = 200,
                evolution_steps = 15,
                mutation_factor = 0.2,
                dieback_rate = 0.8,
                temperature = 0.2,
                crossingover = random_crossingover,
                construct_loop = random_construct_loop,
                mutation = random_mutation,
            )
            dump_solution(solution, score, file_vrp.with_suffix(".msol"))
        except:
            continue

100%|██████████| 63/63 [07:31<00:00,  7.16s/it, data/E/E-n23-k3.vrp]  


### Анализ

#### Посчитаем разницу времени исполнения

In [198]:
calc_best_solution(
    data_path = "data/E/E-n101-k14.vrp",
    population_size = 200,
    evolution_steps = 15,
    mutation_factor = 0.2,
    dieback_rate = 0.8,
    temperature = 0.2,
    crossingover = random_crossingover,
    construct_loop = random_construct_loop,
    mutation = random_mutation,
    use_tqdm=True,
)

100%|██████████| 15/15 [00:27<00:00,  1.84s/it, best fit: 1865.3804720126209]


([array([67, 60, 83,  8, 36, 46, 45, 17, 84, 16, 61]),
  array([39, 44, 98, 93,  5,  6]),
  array([25, 54, 12, 76, 81, 35, 34]),
  array([30, 65,  9, 78, 24, 56, 22, 53]),
  array([ 7, 19, 47, 82, 99, 59, 40]),
  array([85, 37, 97, 87, 13]),
  array([ 50,  51,  79,  77,  21,  73, 100,  96]),
  array([69, 72, 57, 42, 15, 43, 38, 91, 92, 31]),
  array([86, 14, 41, 23, 74,  2]),
  array([62, 10, 66, 80, 75, 95]),
  array([26,  4, 55, 68,  3]),
  array([28, 27, 63, 64, 49, 94]),
  array([89, 11, 90, 32, 70,  1, 33, 58]),
  array([29, 71, 20, 88, 48, 18, 52])],
 1859.866999536081)

In [199]:
calc_best_solution(
    data_path = "data/E/E-n22-k4.vrp",
    population_size = 200,
    evolution_steps = 15,
    mutation_factor = 0.2,
    dieback_rate = 0.8,
    temperature = 0.2,
    crossingover = random_crossingover,
    construct_loop = random_construct_loop,
    mutation = random_mutation,
    use_tqdm=True,
)

100%|██████████| 15/15 [00:04<00:00,  3.17it/s, best fit: 435.4796275290968]


([array([14, 17, 21, 19, 13]),
  array([16, 20, 18, 15]),
  array([ 2,  1,  3,  4,  8, 11]),
  array([10,  6,  5,  7,  9, 12])],
 404.1839155552904)

In [207]:
calc_best_solution(
    data_path = "data/B/B-n78-k10.vrp",
    population_size = 200,
    evolution_steps = 15,
    mutation_factor = 0.2,
    dieback_rate = 0.8,
    temperature = 0.2,
    crossingover = random_crossingover,
    construct_loop = random_construct_loop,
    mutation = random_mutation,
    use_tqdm=True,
)

100%|██████████| 15/15 [00:29<00:00,  1.98s/it, best fit: 2183.7034685637395]


([array([52, 47, 27, 63, 55, 74, 28]),
  array([ 9, 39, 54, 20, 31, 32, 51, 58,  1]),
  array([25, 66, 60, 41, 48, 30, 37, 75, 15, 22]),
  array([44, 71, 46, 45, 57, 49]),
  array([34, 73, 29, 61, 50,  6, 18]),
  array([ 2,  3, 76, 12, 69]),
  array([40, 10, 14, 36, 59, 35,  4, 62]),
  array([17, 24, 21, 56, 65,  7, 33, 13, 19, 42, 11, 77]),
  array([ 8, 70,  5, 16, 68, 26, 67]),
  array([23, 64, 53, 72, 38, 43])],
 2131.45595197623)

In [208]:
calc_best_solution(
    data_path = "data/B/B-n31-k5.vrp",
    population_size = 200,
    evolution_steps = 15,
    mutation_factor = 0.2,
    dieback_rate = 0.8,
    temperature = 0.2,
    crossingover = random_crossingover,
    construct_loop = random_construct_loop,
    mutation = random_mutation,
    use_tqdm=True,
)

100%|██████████| 15/15 [00:10<00:00,  1.39it/s, best fit: 719.2490225541349]


([array([29,  6, 13,  8,  2, 10, 27, 20]),
  array([ 9,  1, 19, 24, 15, 14, 11]),
  array([16, 18,  5, 21]),
  array([12, 23, 30, 26, 28]),
  array([ 4, 22,  7, 17,  3, 25])],
 719.1191008813632)

In [209]:
calc_best_solution(
    data_path = "data/A/A-n80-k10.vrp",
    population_size = 200,
    evolution_steps = 15,
    mutation_factor = 0.2,
    dieback_rate = 0.8,
    temperature = 0.2,
    crossingover = random_crossingover,
    construct_loop = random_construct_loop,
    mutation = random_mutation,
    use_tqdm=True,
)

100%|██████████| 15/15 [00:31<00:00,  2.11s/it, best fit: 2716.4560687426138]


([array([24, 68, 16, 43,  8, 48, 52,  7, 21, 40]),
  array([49, 73, 38, 58, 54, 62, 71, 14, 10]),
  array([77,  6, 78, 27, 29,  3, 66, 42]),
  array([50, 45, 22, 32, 36, 64, 60]),
  array([59, 20, 26, 19, 46, 74, 12]),
  array([13, 44, 39, 33,  9, 53]),
  array([72,  4, 55, 69, 65, 35, 37, 18, 34, 30, 31, 67, 70]),
  array([76, 41, 75, 61,  5,  1]),
  array([63, 23,  2, 47, 56, 15, 25, 51]),
  array([79, 28, 11, 17, 57])],
 2716.4560687426138)

In [210]:
calc_best_solution(
    data_path = "data/A/A-n32-k5.vrp",
    population_size = 200,
    evolution_steps = 15,
    mutation_factor = 0.2,
    dieback_rate = 0.8,
    temperature = 0.2,
    crossingover = random_crossingover,
    construct_loop = random_construct_loop,
    mutation = random_mutation,
    use_tqdm=True,
)

100%|██████████| 15/15 [00:11<00:00,  1.28it/s, best fit: 1071.298614413866] 


([array([17, 19, 31, 12, 30]),
  array([26, 14, 24, 27]),
  array([16,  7, 28, 18,  9, 15, 29, 20]),
  array([ 8, 11, 23,  6, 13, 21,  1]),
  array([ 3,  2,  4, 22, 10, 25,  5])],
 1056.6659040413006)

In [29]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

def load_results(results_file):
    with open(results_file, "r") as f:
        lines = f.readlines()
        cost = int(lines[-1].split(" ")[1])
        # solution = [np.array(list(map(int, ln.split(": ")[1].split(" ")))) for ln in lines[:-1]]
    return cost

In [30]:
pbar = tqdm(Path("data/").rglob("*.msol"), total = len(list(Path("data/").rglob("*.msol"))))
agg_results = []
index = []
for file_vrp in pbar:
    agg_results.append({
        "our_sol": load_results(file_vrp),
        "sol": load_results(file_vrp.with_suffix(".sol")),
    })
    index.append(file_vrp.stem)

agg_results = pd.DataFrame(agg_results, index=index)
agg_results.head()

100%|██████████| 61/61 [00:00<00:00, 16804.76it/s]


,our_sol,sol
B-n45-k5,1146,751
B-n44-k7,1203,909
B-n45-k6,929,678
B-n43-k6,971,742
B-n35-k5,1200,955


In [33]:
diff = np.abs(agg_results["our_sol"] - agg_results["sol"])

for letter in ["A", "B", "E"]:
    print(letter)
    print(diff[diff.index.str.startswith(letter)].describe())

A
count     27.000000
mean     426.555556
std      186.657097
min      166.000000
25%      271.500000
50%      380.000000
75%      581.000000
max      837.000000
dtype: float64
B
count     23.000000
mean     410.000000
std      202.084366
min       80.000000
25%      272.500000
50%      395.000000
75%      524.500000
max      958.000000
dtype: float64
E
count     11.000000
mean     349.909091
std      244.711444
min       38.000000
25%      106.500000
50%      470.000000
75%      547.500000
max      719.000000
dtype: float64
